In [1]:
from MovistarClient import MovistarClient
import httpx
import pandas as pd
from movi_payload_decode import decode_payload, encode_payload
import json
from datetime import datetime
import requests

In [6]:
mClient = MovistarClient()
aclient = httpx.AsyncClient()

In [112]:
               #CAMBIAR CEDULAS POR CADA PETICION
res = await mClient.get_clients(aclient, info_list= ['0245495'], info_rule='contiene')

0245495 id_number contiene


In [113]:
clients_df = pd.DataFrame(columns=['custumerId', 'name', 'id_number', 'phone_number', 'phone_status', 'contextId'])
for r in res:
    body = json.loads(r.json()['responses'][0]['body'])
    for key, value in body['data'].items():
        name = value['data']['-1']['v']['text']
        id_number = value['data']['9144718640113557588']['v']
        nombers = value['data']['9145139830513734387']
        custumerId = key[-20:-1]
        if nombers:
            for phone_object in nombers['v']:
                phone_number = phone_object['text']
                phone_status = phone_object['line_status_text']
                contextId = phone_object['line_id'][-13:len(phone_object['line_id'])]
                client_row = pd.DataFrame({'custumerId':custumerId, 'name':name, 'id_number':id_number, 'phone_number':phone_number, 'phone_status':phone_status, 'contextId':contextId}, index=[0])
                clients_df = pd.concat([clients_df, client_row], ignore_index=True)
        else:
            client_row = pd.DataFrame({'custumerId':custumerId, 'name':name, 'id_number':id_number, 'phone_number':'', 'phone_status':'', 'contextId':''}, index=[0])
            clients_df = pd.concat([clients_df, client_row], ignore_index=True)

In [115]:
custumerIds = clients_df['custumerId'].tolist()
contextIds = clients_df['contextId'].tolist()

In [116]:
rp = await mClient.get_lines_info(aclient, custumerIds, contextIds)

In [162]:
all_dfs = []

for i, response in enumerate(rp):
    resp = response.json()
    try:
        data = resp['data']
    except KeyError:
        continue
    dt = data['com.netcracker.csrdesktop.spa.shared.dto.ContextsDto']['contextTabs']
    fc = data['com.netcracker.solutions.tfnecu.csrd.spa.shared.dto.SubscriberInformationDto']

    fecha_C_linea = fc.get('actualStartDate') # VERIFICAOD

    if len(dt) >= 2:
        dt_combined = {
            'id_tel': dt[1]['id'],  # MOSTRAR PARA REFERENCIA TABLA DE TELEFONOS
            'telefono': dt[1]['title'],
            'Estado_tel': dt[1]['statusText'],
            'id': dt[0]['id'],     # MOSTRAR PARA REFERENCIA TABLA DE TELEFONOS
            'Cuenta de Linea': dt[0]['title'],
            'Estado Linea': dt[0]['statusText'],
            'Fecha inicio': fecha_C_linea,
        }
        df_c = pd.DataFrame([dt_combined])
    else:
        df_c = pd.DataFrame([{
            'id_tel': '',          # MOSTRAR PARA REFERENCIA TABLA DE TELEFONOS
            'telefono': '',
            'Estado_tel': '',
            'id': dt[0]['id'] if dt else '',       # MOSTRAR PARA REFERENCIA TABLA DE TELEFONOS
            'Cuenta de Linea': dt[0]['title'] if dt else '',
            'Estado Linea': dt[0]['statusText'] if dt else '',
            'Fecha inicio': fecha_C_linea,
        }])
    
    all_dfs.append(df_c)
    #df_filter_c = df_c[df_c['Cuenta_linea'].str.contains('Pospago|Prepago', na=False)]
    df = pd.concat(all_dfs, ignore_index=True)

In [164]:
comparar = pd.merge(clients_df, df, left_on='phone_number', right_on='telefono', how='left') # omito custumerId, contextId, telefono(REFERENCIA) y Estado_tel 
mostrar = comparar[[ 'name', 'id_number', 'phone_number', 'phone_status','Cuenta de Linea', 'Estado Linea', 'Fecha inicio']]

In [8]:
comparar = pd.read_csv('comparar.csv', dtype=str)

In [46]:
comparar

,custumerId,name,id_number,phone_number,phone_status,contextId,id_tel,telefono,Estado_tel,id,Cuenta de Linea,Estado Linea,Fecha inicio
0,9153282403313819421,MANUEL MARCELO SACTA SUCUZHAÑAY,0302454954,983713317,Activo,1xn1q4hossij8,1xn1q4hossij8,983713317,Activo,1xn1r9uxbvc9j,Prepago: 46232621,Activo,13/03/2023
1,9153282403313819421,MANUEL MARCELO SACTA SUCUZHAÑAY,0302454954,987765699,Desconectado,1xjivt9e6db67,1xjivt9e6db67,987765699,Desconectado,1xjiwymhrocgu,Prepago: 39731841,Esperando por Terminación,11/02/2019
2,9153282403313819421,MANUEL MARCELO SACTA SUCUZHAÑAY,0302454954,998416311,Desconectado,1xjsmv8z6p8rz,1xjsmv8z6p8rz,998416311,Desconectado,1xjiwymhrocgu,Prepago: 39731841,Esperando por Terminación,06/06/2019
3,9157714354113560669,KLEBER GERMAN CONDOR SINGAUCHO,0502454952,995205858,Desconectado,1xkqit3wasmru,1xkqit3wasmru,995205858,Desconectado,1xkqjygo8jpcn,Prepago: 41085495,Activo,08/07/2020
4,9157714354113560669,KLEBER GERMAN CONDOR SINGAUCHO,0502454952,995339241,Desconectado,1xktr2caidjlx,1xktr2caidjlx,995339241,Desconectado,1xkqjygo8jpcn,Prepago: 41085495,Activo,15/08/2020
5,9149095187413937508,JULIO HERNAN CHALCO DUCHIMAZA,0102454956,958844717,Desconectado,1xmtt4j15fopi,1xmtt4j15fopi,958844717,Desconectado,1xmtu9v4e8xqx,Prepago: 45672864,Activo,10/12/2022
6,9149095187413937508,JULIO HERNAN CHALCO DUCHIMAZA,0102454956,995768548,Desconectado,1xmtn6tc42snw,1xmtn6tc42snw,995768548,Desconectado,1xmtobyfxbvad,Pospago: 45661717,Esperando por Terminación,08/12/2022
7,9149095208013713602,SEGUNDO SAUL BARRAGAN MOREJON,0902454958,962645856,Desconectado,1xjlf08thsf15,1xjlf08thsf15,962645856,Desconectado,1xjlg5lewsi94,Prepago: 39841174,Activo,13/03/2019
8,9149095208013713602,SEGUNDO SAUL BARRAGAN MOREJON,0902454958,979153195,Desconectado,1xjlf0oj31jae,1xjlf0oj31jae,979153195,Desconectado,1xjlg5lewsi94,Prepago: 39841174,Activo,13/03/2019
9,9149095208013713602,SEGUNDO SAUL BARRAGAN MOREJON,0902454958,983726246,Desconectado,1xjlf0phtbw3j,1xjlf0phtbw3j,983726246,Desconectado,1xjlg5lewsi94,Prepago: 39841174,Activo,13/03/2019


In [9]:
res = await mClient.get_custumers_info(aclient, custumerIds=comparar['custumerId'].drop_duplicates().tolist())
res

[<Response [200 OK]>,
 <Response [200 OK]>,
 <Response [200 OK]>,
 <Response [200 OK]>,
 <Response [200 OK]>,
 <Response [200 OK]>,
 <Response [200 OK]>,
 <Response [200 OK]>,
 <Response [200 OK]>,
 <Response [200 OK]>]

In [ ]:
#port out
port_out_df = pd.DataFrame()
for r in res:
    body = json.loads(r.json()['responses'][1]['body'])
    data = body['data']
    if not data:
        continue
    for key, value in data.items():
        text = value['data']['-1']['v']['text']
        numero =value['data']['9145173418813998592']['v'][0]['text']
        estado = value['data']['9145157196513977236']['v']['v']
        df = pd.DataFrame({'name':text, 'phone_number':numero, 'estado':estado}, index=[0])
        port_out_df = pd.concat([port_out_df, df], ignore_index=True)


In [50]:
#port in
port_in_df = pd.DataFrame()
for r in res:
    body = json.loads(r.json()['responses'][0]['body'])
    data = body['data']
    if not data:
        continue
    for key, value in data.items():
        text = value['data']['-1']['v']['text']
        numero =value['data']['9145173418813998592']['v'][0]['text']
        estado = value['data']['9145157196513977236']['v']['v']
        df = pd.DataFrame({'name':text, 'phone_number':numero, 'estado':estado}, index=[0])
        port_in_df = pd.concat([port_in_df, df], ignore_index=True)


In [89]:
hist_df = pd.DataFrame()
for r in res:
    data = json.loads(r.json()['responses'][2]['body'])['data']
    for key, value in data.items():
        try:
            date_ts = int(value['data']['9157884418713223833_9145589430013662246']['v'])/1000
            date = datetime.fromtimestamp(date_ts)
        except KeyError:
            date_ts = None
        try:
            nombre = value['data']['9072741107013063688']['v']['text']
        except KeyError:
            nombre = None
        try:
            number = value['data']['9144543752013303034']['v']['text']
        except KeyError:
            number = None
        
        hist_df = pd.concat([hist_df, pd.DataFrame({'name':nombre, 'phone_number':number, 'date':date}, index=[0])], ignore_index=True)


In [93]:
hist_df.sort_values(by='date', ascending=False).drop_duplicates(subset=['phone_number', 'name'], keep='first').reset_index(drop=True)

,name,phone_number,date
0,PREPAGO TODO EN UNO [992800599],992800599,2024-10-29 16:45:04
1,PREPAGO TODO EN UNO [983713317],983713317,2024-08-18 02:02:22
2,PLAN MOVISTAR STARTER PLUS FS. [995768548],995768548,2024-08-12 01:00:22
3,BONO 2GB PERMANENCIA #1025190666,None,2024-08-12 01:00:22
4,PREPAGO TODO EN UNO [995916522],995916522,2024-06-28 01:25:29
5,PREPAGO TODO EN UNO [995394422],995394422,2024-06-24 22:50:21
6,PREPAGO TODO EN UNO [995614346],995614346,2024-03-05 20:17:32
7,PREPAGO TODO EN UNO [987765699],987765699,2023-11-28 07:07:26
8,PREPAGO TODO EN UNO [995205858],995205858,2023-08-11 06:24:53
9,PREPAGO TODO EN UNO [958844717],958844717,2023-06-12 20:06:21


In [56]:
res[4].json()['responses'][0]['body']

'{"metamodel":{"c":"com.netcracker.platform.ui.plugins.rest.service.transport.metamodel.MetaModel"},"c":"com.netcracker.platform.ui.plugins.rest.service.transport.response.TableCtrlResponse","data":{},"removed_ids":[],"filteringDescriptor":{"c":"com.netcracker.platform.ui.plugins.gwt.tablectrl.shared.transport.filtering.FilteringDescriptor","filters":{}},"root_ids":[],"dataErrors":{},"maxTableSize":1000,"totalRows":0,"message":"","objectPKs":[],"nextPageFirstRow":0,"responseParams":{"widgetContainerId":"9145244403913687320"},"sortingDescriptor":{"c":"com.netcracker.platform.ui.plugins.gwt.tablectrl.shared.transport.SortingDescriptor","sortings":{"4063077486013004590":{"c":"com.netcracker.platform.ui.plugins.gwt.tablectrl.shared.transport.SortingDescriptorItem"}}},"config":{"userViewData":"{\\"options\\":[{\\"isDefault\\":\\"false\\",\\"name\\":\\"default\\",\\"containerId\\":\\"9145244403913687320\\",\\"value\\":\\"\\"},{\\"isDefault\\":\\"true\\",\\"name\\":\\"Solicitudes de Port-In\\

In [21]:
# [0] = Port-Out
# [1] = Port-In
# [2] = Cambiar Historial

# PORT OUT 

solicitud = json.loads(res['responses'][2]['body'])['data']['java.math.BigInteger:{mot:9145441709613629303;mas:1;content:9162837911013567342}']['data']['-1']['v']['text']
numero = json.loads(res['responses'][2]['body'])['data']['java.math.BigInteger:{mot:9145441709613629303;mas:1;content:9162837911013567342}']['data']['9145173418813998592']['v'][0]['text']
fecha = json.loads(res['responses'][2]['body'])['data']['java.math.BigInteger:{mot:9145441709613629303;mas:1;content:9162837911013567342}']['data']['4063077486013004590']['v'] # TIMESTAMP
estado = json.loads(res['responses'][2]['body'])['config']['columns'][4]['presenterConfig']['listValues'][5]['text']

df_Port_out = pd.DataFrame([{'solicitud': solicitud, 'numero': numero, 'fecha': fecha, 'estado': estado}])

df_Port_out


TypeError: list indices must be integers or slices, not str

In [464]:
# [0] = Port-Out
# [1] = Port-In
# [2] = Cambiar Historial

# PORT - IN

solicitud = json.loads(res['responses'][1]['body'])['data']['java.math.BigInteger:{mot:9144992079713808002;mas:1;content:9165851102013691409}']['data']['-1']['v']['text']
numero = json.loads(res['responses'][1]['body'])['data']['java.math.BigInteger:{mot:9144992079713808002;mas:1;content:9165851102013691409}']['data']['9145173418813998592']['v'][0]['text']
fecha = json.loads(res['responses'][1]['body'])['data']['java.math.BigInteger:{mot:9144992079713808002;mas:1;content:9165851102013691409}']['data']['4063077486013004590']['v'] # VERIFICAR - TIMESTAMP
estado = json.loads(res['responses'][1]['body'])['config']['columns'][4]['presenterConfig']['listValues'][5]['text']

df_Port_in = pd.DataFrame([{'solicitud': solicitud, 'numero': numero, 'fecha': fecha, 'estado': estado}])

df_Port_in

'Solicitud común Port-In Órden – 11:17:11 05/02/2023'

In [ ]:
# [0] = Port-Out
# [1] = Port-In
# [2] = Cambiar Historial

# CAMBIAR HISTORIAL 
Cambiar_H = res.json()['responses'][2]['body']

for key, value in json.loads(Cambiar_H)['data'].items():
    date_ts = int(value['data']['9157884418713223833_9145589430013662246']['v']) / 1000
    date = datetime.fromtimestamp(date_ts).strftime('%Y-%m-%d %H:%M:%S')
    message = value['data']['9072741107013063688']['v']['text']
    print(date, message)
    